### Notebook to parse text files to produce cleaned text of RAD decisions

Sean Rehaag

License: Creative Commons Attribution-NonCommercial 4.0 International (CC BY-NC 4.0). 

Dataset & Code to be cited as:

Sean Rehaag, "Refugee Appeal Division Bulk Decisions Dataset" (2023), online: Refugee Law Laboratory <https://refugeelab.ca/bulk-data/rad/>.

Notes:

(1) Data Source: Immigration and Refugee Board. In the Fall of 2022, the IRB added the Refugee Law Laboratory to their email distribution list for legal publishers of RAD decisions. The RLL therefore receives new RAD cases as they are released for publication by the IRB. Also, in the fall of 2022 the Immigration and Refugee Board provided the RLL with a full backlog of approximately 116k published decisions from all divisions (RAD, RPD, ID, IAD). 

(2) Unofficial Data: The data are unofficial reproductions. For official versions, please contact the Immigration and Refugee Board. 

(3) Non-Affiliation / Endorsement: The data has been collected and reproduced without any affiliation or endorsement from the Immigration and Refugee Board.

(4) Non-Commerical Use: As indicated in the license, data may be used for non-commercial use (with attribution) only. For commercial use, please contact the Immigration and Refugee Board. 

(5) Accuracy: Data was collected and processed programmatically for the purposes of academic research. While we make best efforts to ensure accuracy, data gathering of this kind inevitably involves errors. As such the data should be viewed as preliminary information aimed to prompt further research and discussion, rather than as definitive information.

Acknowledgements: Thanks to Rafael Dolores for coding the parsing scripts.


# Installing Libraries

In [1]:
!pip install langdetect
!pip install regex

# Importing Libraries

In [2]:
import os
import regex as re 
import pandas as pd
from datetime import datetime
from langdetect import detect
from difflib import get_close_matches
import json

## Declaring Constant
Here, we specify the directories containing our data files.

In [3]:
DATA_DIRS = ["../RAD Decisions TEXT"] #  "../IRB Decisions - Initial Request - TEXT"]

## Language Detection
This function determines the language of a given text.

In [4]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return None

## Decision Maker Extraction
This function searches the given file for the decision maker using regular expressions.

In [5]:
def extract_decision_maker(content):
    patterns = [
        # String in line immediately after 'Panel' and before 'Tribunal', allowing tabs and spaces
        r"^Panel\s*([^\n]+?)\s*\n\s*Tribunal\s*$",  
        # String in line immediately after 'Tribunal' and before 'Panel', allowing tabs and spaces
        r"^Tribunal\s*([^\n]+?)\s*\n\s*Panel\s*$",
        # String in line immediately after 'Tribunal' and followed by another 'Tribunal', allowing tabs and spaces
        r"^Tribunal\s*([^\n]+?)\s*\n\s*Tribunal\s*$"
    ]

    for pattern in patterns:
        # Use re.MULTILINE to allow ^ and $ to match the start and end of each line
        match = re.search(pattern, content, re.IGNORECASE | re.MULTILINE)
        if match:
            captured = match.group(1).strip()
            # Check if captured group ends with 'Tribunal' or 'Panel' and exclude it
            if not captured.endswith("Tribunal") and not captured.endswith("Panel"):
                return captured
    return None


## Regular Expression Detector
Functions to parse the date from text files while accounting for several different formats

In [6]:
def match_date_patterns(content):
    patterns = {
        "original_decision": (r"Date of decision\s+([\w\s]+),\s+(\d{4})\s+\(original decision\)", lambda m: m.group(1).strip().split() + [m.group(2).strip()]),
        "tribunal": (r"Tribunal\s*\n\s*([\w\s]+?)\s*\n\s*Date of decision", lambda m: m.group(1).replace(',', '').split()),
        "original": (r"Original\s+([\w]+\s+\d{1,2}(?:st|nd|rd|th)?,?\s+\d{4})", lambda m: m.group(1).replace(',', '').split()),
        "primary": (r"Date (?:of decision|de la décision)\s*(?:Le )?\s*((?:(?:\d{1,2}|1er)\s+[\w]+\s*,?\s*\d{1,4})|\w+\s+\d{1,2}(?:st|nd|rd|th)?\s*,?\s*\d{1,4}|\d{1,2}-\d{1,2}-\d{1,4})", lambda m: m.group(1).replace(',', '').split())
    }

    for key, (pattern, process) in patterns.items():
        match = re.search(pattern, content, re.IGNORECASE)
        if match:
            return process(match)

    return None


## Date Formatter
Takes detected regular expression and converts into one common format

In [7]:
def correct_month_name(misspelled_month, possibilities=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], cutoff=0.6):
    correct_months = get_close_matches(misspelled_month, possibilities, n=1, cutoff=cutoff)
    return correct_months[0] if correct_months else misspelled_month


def process_date_parts(parts, french_month_mapping):
    # Function to correct year typos (e.g., '020' to '2020')
    def correct_year_typo(year):
        if len(year) == 3 and year.startswith("0"):
            return "20" + year[1:]
        return year

    if '-' in parts[0]:  # Numeric date format 'dd-mm-yyyy'
        day, month, year = parts[0].split('-')
        year = correct_year_typo(year)
        return datetime(int(year), int(month), int(day)).date().strftime('%Y-%m-%d')

    elif parts[0].isdigit() or parts[0].lower() == '1er':  # Day first, can be French or English
        day = 1 if parts[0].lower() == '1er' else int(parts[0])
        month = parts[1].lower().replace('é', 'e').replace('û', 'u').replace('ô', 'o')
        year = correct_year_typo(parts[2])
        if month in french_month_mapping:
            return datetime(int(year), french_month_mapping[month], day).date().strftime('%Y-%m-%d')
        else:
            corrected_month = correct_month_name(month.capitalize())
            try:
                return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
            except ValueError as e:
                print(f"Error parsing date: {e}")
                return None

    else:  # Month first, English format with possible ordinal suffix
        if len(parts) == 2 and parts[1].isdigit() and len(parts[1]) > 2:
            year_str = parts[1][-4:]
            day_str = parts[1][:-4]
            year = correct_year_typo(year_str)
            day = int(day_str)
        else:
            day = re.sub(r"[^\d]", "", parts[1])  # Remove ordinal suffix if present
            day = int(day) if day.isdigit() else 1  # Default to 1 if day part is not a number
            year = correct_year_typo(parts[2])

        corrected_month = correct_month_name(parts[0].capitalize())
        try:
            return datetime.strptime(f"{corrected_month} {day} {year}", '%B %d %Y').date().strftime('%Y-%m-%d')
        except ValueError as e:
            print(f"Error parsing date: {e}")
            return None

    return None

## Document Date Extraction
This function searches the given file for the document date using regular expressions, taking into account both French and English texts.

In [15]:
def extract_document_date(content):
    french_month_mapping = {
        'janvier': 1, 'fevrier': 2, 'mars': 3, 'avril': 4,
        'mai': 5, 'juin': 6, 'juillet': 7, 'aout': 8,
        'septembre': 9, 'octobre': 10, 'novembre': 11, 'decembre': 12
    }
    
    parts = match_date_patterns(content)
    
    if not parts:
        return None
    return process_date_parts(parts, french_month_mapping)

## File Processor Helpers

In [16]:
def extract_rad_number(content):
    """Extracts the RAD number from the content."""
    for line in content.splitlines():
        if "RAD File" in line:
            rad_number_match = re.search(r"([A-Z]{2}\d+-\d+)", line)
            if rad_number_match:
                return rad_number_match.group(1)
    return None

def process_file(file_path):
    """Processes a single file and extracts data."""
    with open(file_path, 'r', errors='replace') as file:
        content = file.read()

    rad_number = extract_rad_number(content)
    if rad_number:
        lang = detect_language(content)
        decision_maker_name = extract_decision_maker(content)
        document_date = extract_document_date(content)
        year = int(document_date.split('-')[0]) if document_date else None

        return {
            'citation1': rad_number,
            'citation2': '',
            'dataset': 'RAD',
            'name': '',
            'source_url': os.path.basename(file_path),
            'scraped_timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            'document_date': document_date,
            'year': year,
            'unofficial_text': '',
            'language': lang,
            'other': json.dumps({'decision-maker_name': decision_maker_name}, ensure_ascii=False),
        }
    return None

## Processing Files
This block of code reads each file in the dataset directories to extract the needed information, using the previously defined functions and form a Pandas dataframe which is provided in a csv.

In [17]:
# Main data processing loop
data_records = []

for data_dir in DATA_DIRS:
    if os.path.exists(data_dir) and os.path.isdir(data_dir):
        for file_name in os.listdir(data_dir):
            print(f"Parsing file: {file_name}")
            if not file_name.startswith('~'):
                file_path = os.path.join(data_dir, file_name)
                record = process_file(file_path)
                if record:
                    data_records.append(record)

Parsing file: AppToReopenTB8-14702a.txt
Parsing file: AppToReopenTB8-14702tf.txt
Parsing file: Decision concerning G. Bazin (finale version) (amended 5.5.2022) (003) - sanitized.txt
Parsing file: Decision G. Bazin_ francais_modifiee (003) - sanitized.txt
Parsing file: MB7-00112a.txt
Parsing file: MB7-00112tf.txt
Parsing file: MB7-03926f.txt
Parsing file: MB7-03926ta.txt
Parsing file: MB7-24221 f.txt
Parsing file: MB7-24221ta.txt
Parsing file: MB8-01089a.txt
Parsing file: MB8-01089tf.txt
Parsing file: MB8-02514f.txt
Parsing file: MB8-02514ta.txt
Parsing file: MB8-02728.txt
Parsing file: MB8-02728tf.txt
Parsing file: MB8-02935f.txt
Parsing file: MB8-02935ta.txt
Parsing file: MB8-03538a.txt
Parsing file: MB8-03538tf.txt
Parsing file: MB8-04750f.txt
Parsing file: MB8-04750ta.txt
Parsing file: MB8-06936a.txt
Parsing file: MB8-06936tf.txt
Parsing file: MB8-07131f.txt
Parsing file: MB8-07131ta.txt
Parsing file: MB8-08060f.txt
Parsing file: MB8-08060ta.txt
Parsing file: MB8-08280f.txt
Parsing 

Parsing file: MB9-11208f.txt
Parsing file: MB9-11208ta.txt
Parsing file: MB9-11253f.txt
Parsing file: MB9-11253ta.txt
Parsing file: MB9-11259f.txt
Parsing file: MB9-11259ta.txt
Parsing file: MB9-11459f.txt
Parsing file: MB9-11459ta.txt
Parsing file: MB9-11472a.txt
Parsing file: MB9-11472tf.txt
Parsing file: MB9-11602f.txt
Parsing file: MB9-11602ta.txt
Parsing file: MB9-11814a.txt
Parsing file: MB9-11814tf.txt
Parsing file: MB9-11934f.txt
Parsing file: MB9-11934ta.txt
Parsing file: MB9-11939f.txt
Parsing file: MB9-11939te.txt
Parsing file: MB9-11966f.txt
Parsing file: MB9-11966te.txt
Parsing file: MB9-12342f.txt
Parsing file: MB9-12342ta.txt
Parsing file: MB9-12494a.txt
Parsing file: MB9-12494tf.txt
Parsing file: MB9-12499a.txt
Parsing file: MB9-12499tf.txt
Parsing file: MB9-12504ff.txt
Parsing file: MB9-12504ta.txt
Parsing file: MB9-12537f.txt
Parsing file: MB9-12537ta.txt
Parsing file: MB9-12551a.txt
Parsing file: MB9-12551f.txt
Parsing file: MB9-12570a.txt
Parsing file: MB9-12570tf.t

Parsing file: MB9-21125tf.txt
Parsing file: MB9-21127f.txt
Parsing file: MB9-21127ta.txt
Parsing file: MB9-21267f.txt
Parsing file: MB9-21267ta.txt
Parsing file: MB9-21516f.txt
Parsing file: MB9-21516ta.txt
Parsing file: MB9-21654a.txt
Parsing file: MB9-21654tf.txt
Parsing file: MB9-21655a.txt
Parsing file: MB9-21655tf.txt
Parsing file: MB9-21656a.txt
Parsing file: MB9-21656tf.txt
Parsing file: MB9-21661f.txt
Parsing file: MB9-21661ta.txt
Parsing file: MB9-21802a.txt
Parsing file: MB9-21802tf.txt
Parsing file: MB9-21924f.txt
Parsing file: MB9-21924ta.txt
Parsing file: MB9-21926 a.txt
Parsing file: MB9-21926tf.txt
Parsing file: MB9-21935f.txt
Parsing file: MB9-21935ta.txt
Parsing file: MB9-21941f.txt
Parsing file: MB9-21941ta.txt
Parsing file: MB9-22048a.txt
Parsing file: MB9-22048tf.txt
Parsing file: MB9-22375a.txt
Parsing file: MB9-22375tf.txt
Parsing file: MB9-22411a.txt
Parsing file: MB9-22411tf.txt
Parsing file: MB9-22713f.txt
Parsing file: MB9-22713ta.txt
Parsing file: MB9-22724f.

Parsing file: MC0-01410f .txt
Parsing file: MC0-01410ta.txt
Parsing file: MC0-01579 f.txt
Parsing file: MC0-01579ta.txt
Parsing file: MC0-01686f.txt
Parsing file: MC0-01686ta.txt
Parsing file: MC0-01687a.txt
Parsing file: MC0-01687tf.txt
Parsing file: MC0-01705a.txt
Parsing file: MC0-01705tf.txt
Parsing file: MC0-01793 f.txt
Parsing file: MC0-01793ta.txt
Parsing file: MC0-01794 a.txt
Parsing file: MC0-01794tf.txt
Parsing file: MC0-01798a.txt
Parsing file: MC0-01798tf.txt
Parsing file: MC0-01801 a.txt
Parsing file: MC0-01801tf.txt
Parsing file: MC0-01907f.txt
Parsing file: MC0-01907ta.txt
Parsing file: MC0-01913f.txt
Parsing file: MC0-01913ta.txt
Parsing file: MC0-01917 a.txt
Parsing file: MC0-01917tf.txt
Parsing file: MC0-01918f.txt
Parsing file: MC0-01918ta.txt
Parsing file: MC0-02034f .txt
Parsing file: MC0-02034ta.txt
Parsing file: MC0-02054f.txt
Parsing file: MC0-02054ta.txt
Parsing file: MC0-02055a.txt
Parsing file: MC0-02055tf.txt
Parsing file: MC0-02058f.txt
Parsing file: MC0-02

Parsing file: MC0-08808a.txt
Parsing file: MC0-08808tf.txt
Parsing file: MC0-08828a.txt
Parsing file: MC0-08828tf.txt
Parsing file: MC0-08834 a.txt
Parsing file: MC0-08834tf.txt
Parsing file: MC0-08857 a.txt
Parsing file: MC0-08857tf.txt
Parsing file: MC0-08908 a.txt
Parsing file: MC0-08908tf.txt
Parsing file: MC0-08913 f.txt
Parsing file: MC0-08913ta.txt
Parsing file: MC0-08939 a.txt
Parsing file: MC0-08939tf.txt
Parsing file: MC0-08954a.txt
Parsing file: MC0-08954tf.txt
Parsing file: MC0-08972a.txt
Parsing file: MC0-08972tf.txt
Parsing file: MC0-08998 a.txt
Parsing file: MC0-08998tf.txt
Parsing file: MC0-09001a.txt
Parsing file: MC0-09001tf.txt
Parsing file: MC0-09023a.txt
Parsing file: MC0-09023tf.txt
Parsing file: MC0-09027 a.txt
Parsing file: MC0-09027tf.txt
Parsing file: MC0-09038 f.txt
Parsing file: MC0-09038ta.txt
Parsing file: MC0-09063 a.txt
Parsing file: MC0-09063tf.txt
Parsing file: MC0-09072a.txt
Parsing file: MC0-09072tf.txt
Parsing file: MC0-09130a.txt
Parsing file: MC0-

...

Parsing file: TC0-02936tf.txt
Parsing file: TC0-02944 a.txt
Parsing file: TC0-02944tf.txt
Parsing file: TC0-03070a .txt
Parsing file: TC0-03070tf.txt
Parsing file: TC0-03190a.txt
Parsing file: TC0-03190tf.txt
Parsing file: TC0-03195a.txt
Parsing file: TC0-03195tf.txt
Parsing file: TC0-03353a .txt
Parsing file: TC0-03353tf.txt
Parsing file: TC0-03360a.txt
Parsing file: TC0-03360tf.txt
Parsing file: TC0-03367 a.txt
Parsing file: TC0-03367tf.txt
Parsing file: TC0-03681a.txt
Parsing file: TC0-03681tf.txt
Parsing file: TC0-03685a.txt
Parsing file: TC0-03685tf.txt
Parsing file: TC0-03690a .txt
Parsing file: TC0-03690tf.txt
Parsing file: TC0-03695a.txt
Parsing file: TC0-03695tf.txt
Parsing file: TC0-03854a.txt
Parsing file: TC0-03854tf.txt
Parsing file: TC0-03867 a.txt
Parsing file: TC0-03867tf.txt
Parsing file: TC0-04046a.txt
Parsing file: TC0-04046tf.txt
Parsing file: TC0-04048 a.txt
Parsing file: TC0-04048tf.txt
Parsing file: TC0-04051 a.txt
Parsing file: TC0-04051tf.txt
Parsing file: TC0-

Parsing file: TC0-08712tf.txt
Parsing file: TC0-08713 a.txt
Parsing file: TC0-08713tf.txt
Parsing file: TC0-08720a.txt
Parsing file: TC0-08720tf.txt
Parsing file: TC0-08734a.txt
Parsing file: TC0-08734tf.txt
Parsing file: TC0-08743a.txt
Parsing file: TC0-08743tf.txt
Parsing file: TC0-08744a.txt
Parsing file: TC0-08744tf.txt
Parsing file: TC0-08821a.txt
Parsing file: TC0-08821tf.txt
Parsing file: TC0-08914a.txt
Parsing file: TC0-08914tf.txt
Parsing file: TC0-08985a.txt
Parsing file: TC0-08985tf.txt
Parsing file: TC0-08992a.txt
Parsing file: TC0-08992tf.txt
Parsing file: TC0-09010 a.txt
Parsing file: TC0-09010tf.txt
Parsing file: TC0-09011 a.txt
Parsing file: TC0-09011tf.txt
Parsing file: TC0-09018 a.txt
Parsing file: TC0-09018tf.txt
Parsing file: TC0-09076 a.txt
Parsing file: TC0-09076tf.txt
Parsing file: TC0-09082a.txt
Parsing file: TC0-09082tf.txt
Parsing file: TC0-09131 a.txt
Parsing file: TC0-09131tf.txt
Parsing file: TC0-09139a.txt
Parsing file: TC0-09139tf.txt
Parsing file: TC0-09

Parsing file: TC0-11737tf.txt
Parsing file: TC0-11775 a.txt
Parsing file: TC0-11775tf.txt
Parsing file: TC0-11777 a.txt
Parsing file: TC0-11777tf.txt
Parsing file: TC0-11826 a.txt
Parsing file: TC0-11826tf.txt
Parsing file: TC0-11831 a.txt
Parsing file: TC0-11831tf.txt
Parsing file: TC0-11847 a.txt
Parsing file: TC0-11847tf.txt
Parsing file: TC0-11886 a.txt
Parsing file: TC0-11886 tf.txt
Parsing file: TC0-11936 a.txt
Parsing file: TC0-11936tf.txt
Parsing file: TC0-11938 a.txt
Parsing file: TC0-11938tf.txt
Parsing file: TC0-11972 a.txt
Parsing file: TC0-11972tf.txt
Parsing file: TC0-12030 a.txt
Parsing file: TC0-12030tf.txt
Parsing file: TC0-12032 a.txt
Parsing file: TC0-12032tf.txt
Parsing file: TC0-12055 a.txt
Parsing file: TC0-12055tf.txt
Parsing file: TC0-12182 a.txt
Parsing file: TC0-12182tf.txt
Parsing file: TC0-12185 a.txt
Parsing file: TC0-12185tf.txt
Parsing file: TC0-12198  a.txt
Parsing file: TC0-12198tf.txt
Parsing file: TC0-12261 a.txt
Parsing file: TC0-12261tf.txt
Parsing 

Parsing file: TC1-10106 a.txt
Parsing file: TC1-10106tf.txt
Parsing file: TC1-10177 a.txt
Parsing file: TC1-10177tf.txt
Parsing file: TC1-10937 a.txt
Parsing file: TC1-10937TF.txt
Parsing file: TC1-11196 a.txt
Parsing file: TC1-11196tf.txt
Parsing file: TC1-11198 a.txt
Parsing file: TC1-11198tf.txt
Parsing file: TC1-11703f.txt
Parsing file: TC1-11703ta.txt
Parsing file: TC1-12204 a.txt
Parsing file: TC1-12204tf.txt
Parsing file: TC1-12315 a.txt
Parsing file: TC1-12315tf.txt
Parsing file: TC1-12335 E.txt
Parsing file: TC1-12335 F.txt
Parsing file: TC1-13012 a.txt
Parsing file: TC1-13012tf.txt
Parsing file: TC1-14698 a.txt
Parsing file: TC1-14698tf.txt
Parsing file: TC1-15539 a.txt
Parsing file: TC1-15539tf.txt
Parsing file: TC1-16570 a.txt
Parsing file: TC1-16570tf.txt
Parsing file: TC1-17552 f.txt
Parsing file: TC1-17552ta.txt
Parsing file: TC1-18417 a.txt
Parsing file: TC1-18417tf.txt
Parsing file: TC1-21863 a.txt
Parsing file: TC1-21863tf.txt
Parsing file: TC2-07061 a.txt
Parsing fil

Parsing file: VC0-00211tf.txt
Parsing file: VC0-00213 a.txt
Parsing file: VC0-00213tf.txt
Parsing file: VC0-00250a.txt
Parsing file: VC0-00250tf.txt
Parsing file: VC0-00308 f.txt
Parsing file: VC0-00308 ta.txt
Parsing file: VC0-00311 F.txt
Parsing file: VC0-00311ta.txt
Parsing file: VC0-00364 f.txt
Parsing file: VC0-00364 ta.txt
Parsing file: VC0-00452 a.txt
Parsing file: VC0-00452tf.txt
Parsing file: VC0-00455 a.txt
Parsing file: VC0-00455tf.txt
Parsing file: VC0-00460 a.txt
Parsing file: VC0-00460tf.txt
Parsing file: VC0-00462 a.txt
Parsing file: VC0-00462tf.txt
Parsing file: VC0-00465 a.txt
Parsing file: VC0-00465tf.txt
Parsing file: VC0-00471 a.txt
Parsing file: VC0-00471tf.txt
Parsing file: VC0-00547 a.txt
Parsing file: VC0-00547tf.txt
Parsing file: VC0-00549 a.txt
Parsing file: VC0-00549tf.txt
Parsing file: VC0-00555 a.txt
Parsing file: VC0-00555tf.txt
Parsing file: VC0-00587 a.txt
Parsing file: VC0-00587tf.txt
Parsing file: VC0-00614a.txt
Parsing file: VC0-00614tf.txt
Parsing fi

## Placing output in CSV file

In [14]:
df = pd.DataFrame(data_records)
df['document_date'] = pd.to_datetime(df['document_date']).dt.strftime('%Y-%m-%d')
df.to_csv('output.csv', index=False, encoding='utf-8-sig')